In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import numpy as np
from time import time
import uuid
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import time
import math
%matplotlib inline 


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [4]:
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model = load_model('model3.h5')


Using cache found in C:\Users\m7mde/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-2-25 torch 1.10.2+cu113 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 


In [9]:
def detect(yolo,img):
    results = yolo(img)
    labels, cord,wh = results.xyxyn[0][:, -1], results.xywhn[0][:, :-1],results.xyxyn[0][:, :-1]
    classes = yolo.names
    n = len(labels)
    x_shape, y_shape = img.shape[1], img.shape[0]
    players = []
   
    for i in range(n):
        row = cord[i]
        jj = wh[i]
        if row[4] and jj[4] >= 0.1:
            x, y, w, h = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
            x1,y1,x2,y2 = int(jj[0]*x_shape), int(jj[1]*y_shape), int(jj[2]*x_shape), int(jj[3]*y_shape)
            
            label = classes[int(labels[i])]
            if label == 'person':
                new = img[y1:y2,x1:x2]
                players.append([x1,y1,w,h])
            if label == 'sports ball':
                cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (0,0,0), 2)
    return img,players

def plane(cords_in_plane,players_frame,players_cords,frame,model):
    coptemp=cv2.imread('plane.jpg')
    pts1 = np.float32([[530,286],[530,101],[133,101],[133,286]])
    pts2 = np.float32([[1708,398],[1405,146],[513,146],[212,398]])
    matrix = cv2.getPerspectiveTransform(pts2,pts1)
    
    for i in range(len(cords_in_plane)):
        pts3 = np.float32([cords_in_plane[i]])
        pts3o=cv2.perspectiveTransform(pts3[None, :, :],matrix)
        x1=int(pts3o[0][0][0])
        y1=int(pts3o[0][0][1])
        pp=(x1,y1)
        player = players_frame[i]
        try:
            player = cv2.resize(player,(200,200))
            player = np.expand_dims(player,axis=0)
            val = model.predict(player)
            val = np.argmax(val)
            if val == 0:
                cv2.circle(coptemp,pp, 7, (0,0,255),-1)
                cv2.rectangle(frame, (players_cords[i][0], players_cords[i][1]), (players_cords[i][0]+players_cords[i][2], players_cords[i][1]+players_cords[i][3]), (0,0,255), 1)
            elif val == 1:
                cv2.circle(coptemp,pp, 7, (255,0,0),-1)
                cv2.rectangle(frame, (players_cords[i][0], players_cords[i][1]), (players_cords[i][0]+players_cords[i][2], players_cords[i][1]+players_cords[i][3]), (255,0,0), 1)
            else:
                pass
        except:
            continue
        cv2.rectangle(frame, (players_cords[i][0], players_cords[i][1]), (players_cords[i][0]+players_cords[i][2], players_cords[i][1]+players_cords[i][3]), (255,0,0), 1)
        cv2.circle(coptemp,pp, 7, (255,0,0),-1)

    return coptemp,frame

In [10]:
cap = cv2.VideoCapture('test22.mp4')

number_of_splits = 1
while True:
    ret,frame = cap.read()
    players_cords = []
    players_frame = [] 
    cords_in_plane = []
    if ret:
        cropped = frame[::,::]
        h,w,_ = cropped.shape
        cropped_w = int(w/number_of_splits)
        for i in range(number_of_splits):
            new = cropped[::,i*cropped_w:(i+1)*cropped_w]
            new,player = detect(yolo,new)
            for j in player:
                j[0] += i*cropped_w
                players_cords.append(j)
                
        for i in players_cords:
            player = frame[i[1]:i[1]+i[3],i[0]:i[0]+i[2]]
            players_frame.append(player)
            cords = [int((i[0]+i[0]+i[2])/2) ,int((i[1]+i[3]+i[1]+i[3])/2)]
            cords_in_plane.append(cords)
        coptemp,frame = plane(cords_in_plane,players_frame,players_cords,frame,model)
        coptemp = cv2.resize(coptemp,(300,200))
        frame[h-200:h,810:1110] = coptemp
        cv2.imshow('frame',frame)
        if cv2.waitKey(1)==27:
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()